# Progetto: Excel per HR

#### Inizializzazioni
- Leggere i nomi delle cartelle, file, fogli e colonne che mi servono.
- Eliminare i fogli che non mi servono.

#### Lettura input utente:
- Leggere la data come input()

#### Controlli sui nomi e ricerche dei riferimenti
- Controllare che i fogli e le colonne indicate esistano.

#### Foglio "Candidati":
- Per ogni riga (compilata) del foglio:
    - Se "Data invio CV al cliente" == data da input utente:
        - Annotarsi il campo "Id candidato" (anagskill_ids).
    - Altrimenti:
        - Annotarsi di eliminare la riga.
- Se non è stato trovato almeno un candidato:
    - Informare l'utente che non sono stati trovati candidati e non si può procedere.
- Eliminare le righe che non corrispondevano al filtro.

#### Foglio "AnagSkill"
- Per ogni riga (compilata) del foglio:
    - Se "Progr Interno" è tra  gli "Id candidato" (anagskill_ids) annotati nel punto precedente:
        - Annotarsi i campi "Cognome" e "Nome" del candidato selezionato.
    - Altrimenti:
        - Annotarsi di eliminare la riga.
- Eliminare le righe che non corrispondevano al filtro.

#### Creazione file XLSX di output
- Salvare il file XLSX modificato (con le celle e i fogli eliminati).
- Informare l'utente che il file XLSX è stato creato.

#### Cartella con i CV:
- Nella cartella indicata, cerca tutti i file di CV che corrispondono alle coppie "Nome" e "Cognome" dei candidati selezionati.
- Per ciascun nominativo selezionato:
    - Se non è stato trovato alcun file:
        - Informa l'utente che non è stato trovato il file di CV per quel certo candidato e che non si può procedere.
    - Se è stato trovato un solo file:
        - Si annota il file trovato.
    - Altrimenti (se sono stati trovati più file):
        - Informa l'utente che sono stati trovati più file e che non si può procedere.
- Informare l'utente dei CV che sono stati trovati.

#### Creazione file ZIP di output:
- Zippare tutti i file CV trovati.
- Informare l'utente che il file ZIP è stato creato.

In [9]:
import datetime
from pathlib import Path
from zipfile import ZipFile
import openpyxl

#=================================
# INITS
#=================================
# Inizializzo i percorsi
book = openpyxl.load_workbook('./DB C-Lab (HRR).xlsx', data_only=True)
cv_path = Path('./CV al Cliente/')
output_path = Path('./outputs/')
output_xlsx = 'DB C-Lab (Transfer) %s.xlsx'
output_zip = '%s_CVs.zip'

# Rimuovo i fogli non necessari
book.remove(book['Richieste'])
book.remove(book['KPI HRR'])
book.remove(book['TabInt'])

# Inizializzo i nomi
candidati_nome_foglio = 'Candidati'
anagskill_nome_foglio = 'AnagSkill'

candidati_data_invio_col_label = 'Data invio CV al cliente'
candidati_data_invio_col_num = None

candidati_anagskill_id_col_label = 'Id candidato'
candidati_anagskill_id_col_num = None

anagskill_id_col_label = 'Progr Interno'
anagskill_id_col_num = None

anagskill_cognome_col_label = 'Cognome'
anagskill_cognome_col_num = None

anagskill_nome_col_label = 'Nome'
anagskill_nome_col_num = None

#=================================
# USER INPUT
#=================================
while True:
    # Leggo la data come input dall'utente
    data_invio = input('Inserire la "Data invio CV al cliente" in formato dd/mm/yyyy '
                    'e premere Invio.\nPer usare la data odierna, premere '
                    'semplicemente Invio: ')

    # se l'utente ha inserito la data di invio
    if data_invio:
        try:
            # la converte in data inserita in oggetto data
            target_date = datetime.datetime.strptime(data_invio, '%d/%m/%Y').date()
        except ValueError:
            print('La data non è stata in modo corretto. Si prega di riprovare.')
            continue
    # se l'utente ha premuto invio senza inserire la data
    else:
        # usa la data di oggi
        # target_date = datetime.date.today()
        # @TEST: usa la data di test
        target_date = datetime.date.fromisoformat('2023-05-15')
    break

#=================================
# CHECK & MATCH - SHEETS & COLUMNS
#=================================
for nome_foglio in [candidati_nome_foglio, anagskill_nome_foglio]:
    # controllo che il foglio esista
    if nome_foglio not in book.sheetnames:
        raise ValueError(f'Il foglio "{nome_foglio}" non è stato trovato nella '
                        'cartella di lavoro.')

# Apro i fogli che mi servono
foglio_candidati = book[candidati_nome_foglio]
foglio_anagskill = book[anagskill_nome_foglio]

# individuo il numero di colonna della data oggetto del filtro
for col_num in range(1, foglio_candidati.max_column + 1):
    column_name = foglio_candidati.cell(1, col_num).value
    if column_name == candidati_data_invio_col_label:
        candidati_data_invio_col_num = col_num
    elif column_name == candidati_anagskill_id_col_label:
        candidati_anagskill_id_col_num = col_num

# individuo il numero di colonna dell'id oggetto del filtro
for col_num in range(1, foglio_anagskill.max_column + 1):
    column_name = foglio_anagskill.cell(1, col_num).value
    if column_name == anagskill_id_col_label:
        anagskill_id_col_num = col_num
    elif column_name == anagskill_cognome_col_label:
        anagskill_cognome_col_num = col_num
    elif column_name == anagskill_nome_col_label:
        anagskill_nome_col_num = col_num

def check_col(check, label, foglio):
    if not check:
        raise ValueError(f'La colonna "{label}" non è stata trovata nel '
                         f'foglio "{foglio}".')

# controllo che la colonna con la data oggetto del filtro esista nel foglio Candidati
check_col(candidati_data_invio_col_num, candidati_data_invio_col_label, candidati_nome_foglio)
# controllo che la colonna con l'id di AnagSkill esista nel foglio Candidati
check_col(candidati_anagskill_id_col_num, candidati_anagskill_id_col_label, candidati_nome_foglio)
# controllo che la colonna con l'id (progressivo) esista nel foglio AnagSkill
check_col(anagskill_id_col_num, anagskill_id_col_label, anagskill_nome_foglio)
# controllo che la colonna con il cognome esista nel foglio AnagSkill
check_col(anagskill_cognome_col_num, anagskill_cognome_col_label, anagskill_nome_foglio)
# controllo che la colonna con il nome esista nel foglio AnagSkill
check_col(anagskill_nome_col_num, anagskill_nome_col_label, anagskill_nome_foglio)

#=================================
# CANDIDATI
#=================================
anagskill_ids = []
righe_candidati_da_eliminare = []

# per ciascuna riga
for row_num in range(2, foglio_candidati.max_row + 1):
    # se la prima cella contiene dei dati (perché il foglio è sporco e .max_row non funziona bene)
    if foglio_candidati.cell(row_num, 1).value:
    # if all(cell.value is None for cell in foglio_candidati[row_num-1]):
        # ottiene il valore del campo con la data target
        date_cell = foglio_candidati.cell(row_num, candidati_data_invio_col_num).value
        # se il valore ottenuto è di tipo data e se la data corrisponde a quella target
        if isinstance(date_cell, datetime.datetime) and date_cell.date() == target_date:
            # si annota l'ID dell'AnagSkill
            anagskill_id = foglio_candidati.cell(row_num, candidati_anagskill_id_col_num).value
            anagskill_ids.append(anagskill_id)
        else:
            # si annota di cancellare l'intera riga
            righe_candidati_da_eliminare.append(row_num)

# Se non sono stati trovati candidati non procede oltre e informa l'utente
if len(anagskill_ids) == 0:
    raise LookupError('Non sono stati trovati candidati per la data indicata: '
                      f"{target_date.strftime('%d/%m/%Y')}. Esegui di nuovo lo "
                      "script e prova a inserire un'altra data.")

# elimino le righe dei candidati da eliminare
for idx, row_num in enumerate(righe_candidati_da_eliminare):
    foglio_candidati.delete_rows(row_num - idx)

#=================================
# ANAGSKILL
#=================================
righe_anagskill_da_eliminare = []
nominativi_cv = []

# per ciascuna riga
for row_num in range(2, foglio_anagskill.max_row + 1):
    # se la seconda cella contiene dei dati
    # (perché il foglio è sporco e .max_row non funziona bene)
    if foglio_anagskill.cell(row_num, 2).value:
    # if all(cell.value is None for cell in foglio_anagskill[row_num-1]):
        # ottiene il valore del campo con l'id anagskill
        anagskill_id = foglio_anagskill.cell(row_num, anagskill_id_col_num).value
        # se il valore ottenuto è un id di quelli selezionati
        if anagskill_id in anagskill_ids:
            # si annota una tupla (cognome, nome)
            cognome = foglio_anagskill.cell(row_num, anagskill_cognome_col_num).value
            nome = foglio_anagskill.cell(row_num, anagskill_nome_col_num).value
            nominativi_cv.append((cognome, nome))
        else:
            # si annota di cancellare l'intera riga
            righe_anagskill_da_eliminare.append(row_num)

# elimino le righe dei candidati da eliminare
for idx, row_num in enumerate(righe_anagskill_da_eliminare):
    foglio_anagskill.delete_rows(row_num - idx)

#=================================
# WRITE XLSX
#=================================
# preparo il nome per il file di output
output_xlsx = output_xlsx % target_date.isoformat()
output_xlsx_path = output_path / output_xlsx
# salvo il file di output
book.save(output_xlsx_path)

print(f'File XLSX generato in: {output_xlsx_path}')

#=================================
# CV SEARCH
#=================================
# cerco i cv nella cartella CV
# estensioni_cv = ['.pdf', '.docx']
file_list = []
for cognome, nome in nominativi_cv:
    glob_pattern = f'*{cognome} {nome}*'
    cv_trovati = list(cv_path.glob(glob_pattern))
    if len(cv_trovati) == 0:
        raise FileNotFoundError(f'Non è stato trovato il file del CV per la '
                f'persona {cognome} {nome} nella cartella {cv_path}.'
                'Aggiungere il file mancante ed eseguire nuovamente lo script.')
    if len(cv_trovati) == 1:
        file_list += cv_trovati
    else:
        raise FileExistsError(f'Sono stati trovati più file di CV per la '
                f'persona {cognome} {nome} nella cartella {cv_path}.\n'
                'Risolvere il conflitto ed eseguire nuovamente lo script.')
    
print('File di CV trovati:', *[file.name for file in file_list], sep='\n  - ')

#=================================
# CV ZIP
#=================================
# compongo il nome del file zip
output_zip = output_zip % target_date.isoformat()
# creo lo zip con i CV
output_zip_path = output_path / output_zip
with ZipFile(output_zip_path, 'w') as new_archive:
    for file in file_list:
        new_archive.write(file, file.name)

print(f'File ZIP CV generato in: {output_zip_path}')

File XLSX generato in: outputs/DB C-Lab (Transfer) 2023-05-15.xlsx
File di CV trovati:
  - Cognome 2 Nome2.pdf
  - Cognome 9 Nome9.pdf
File ZIP CV generato in: outputs/2023-05-15_CVs.zip
